# Numerical SOS decomposition for $\beta_T$

## Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ncpol2sdpa as ncp
import pandas as pd

# Define quantum operators
A_config = [2,2]
B_config = [2,2]

# Operators in problem
A = [Ai for Ai in ncp.generate_measurements(A_config, 'A')]
B = [Bj for Bj in ncp.generate_measurements(B_config, 'B')]

ops = ncp.flatten([A,B])        # Base monomials involved in problem
subs = {}

# 1. Unitarity: A^2 = 1
for op in ops:
    subs[op**2] = 1

# 2. Commutation: force a canonical ordering B*A → A*B
for Ai in ncp.flatten(A):
    for Bj in ncp.flatten(B):
        subs[Bj * Ai] = Ai * Bj

A0 = A[0][0]
A1 = A[1][0]
B0 = B[0][0]
B1 = B[1][0]

## Testing Victor's Bell expressions at different levels of the hierarchy

In [3]:
def test_point_quantum_bound(r0, r1, level=0, verbose=False):
    """
    Tests whether a point (parametrized by theta, r0 = p1, r1 = 2*p4) is quantum-realizable
    by solving the associated SDP relaxation at level 0 with different sets of monomials.

    Parameters:
    -----------
    theta : float
        The measurement angle θ in radians.
    r0 : float
        Parameter such that p1 = r0.
    r1 : float
        Parameter such that p4 = 0.5 * r1.
    level : str or int
        Level of the relaxation: 0, 'AB', 'ABB', or 'AAB'.
    solver : str
        SDP solver to use (e.g., "mosek", "cvxopt", "sdpa").
    verbose : bool
        Whether to print verbose SDP solver output.

    Returns:
    --------
    primal : float
        The negative primal bound.
    dual : float
        The negative dual bound.
    status : str
        The solver status string.
    """

    obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

    def get_extra_monomials(level):
        """
        Generate extra monomials for a given level string such as:
        'AA', 'BB', 'AB', 'AAB', 'ABB', 'AA+BB+AB+AAB+ABB', etc.
        """
        if level in [0, '0', None]:
            return []

        level_parts = level.split('+') if isinstance(level, str) else []
        monos = []
        Aflat = ncp.flatten(A)
        Bflat = ncp.flatten(B)

        if 'AA' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    monos.append(a1 * a2)

        if 'BB' in level_parts:
            for b1 in Bflat:
                for b2 in Bflat:
                    monos.append(b1 * b2)

        if 'AB' in level_parts:
            for a in Aflat:
                for b in Bflat:
                    monos.append(a * b)

        if 'AAB' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    for b in Bflat:
                        monos.append(a1 * a2 * b)

        if 'ABB' in level_parts:
            for a in Aflat:
                for b1 in Bflat:
                    for b2 in Bflat:
                        monos.append(a * b1 * b2)

        return monos[:]

    op_eqs = []
    op_ineqs = []
    moment_eqs = []
    moment_ineqs = []
    extra_monos = get_extra_monomials(level)

    # Create and solve SDP relaxation at fixed level=0
    sdp = ncp.SdpRelaxation(ops, verbose=int(verbose), normalized=True, parallel=0)
    sdp.get_relaxation(
        level=0,
        equalities= op_eqs[:],
        inequalities= op_ineqs[:],
        momentequalities= moment_eqs[:],
        momentinequalities=moment_ineqs[:],
        objective=-obj,
        substitutions=subs,
        extramonomials=extra_monos
    )
    sdp.solve(solver="mosek")

    return -sdp.primal, -sdp.dual, sdp.status

In [4]:
r0 = 1 - 1/np.sqrt(2)
r1 = 0

for lvl in [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'BB+AB+ABB']:
    p, d, s = test_point_quantum_bound(r0, r1, level=lvl)
    print(f"Level {lvl}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Level 0: primal = 1.0857864377, dual = 1.0857864365, status = optimal
Level AA: primal = 1.0857864379, dual = 1.0857864367, status = optimal
Level BB: primal = 1.0857864383, dual = 1.0857864346, status = optimal
Level AB: primal = 1.0091153569, dual = 1.0091153431, status = optimal
Level BB+AB: primal = 1.0091153531, dual = 1.0091153512, status = optimal
Level ABB: primal = 1.0857864420, dual = 1.0857864355, status = optimal
Level AAB: primal = 1.0828606998, dual = 1.0828606844, status = optimal
Level AB+ABB: primal = 1.0000000084, dual = 0.9999999940, status = optimal
Level BB+AB+ABB: primal = 1.0000000014, dual = 0.9999999990, status = optimal


## Extracting the certificate at $T_{1+A+B+AB+ABB}$

We are able to reach $\beta_T$ at this level of the hierarchy.

In [10]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = [B0 + 1, B1 - 1]          # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials


r0 = 1 - 1/np.sqrt(2) # p1 = r0
r1 = 0 # p4 = 1/2 r1

obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")

if sdp.primal is not None and sdp.dual is not None:
    print(-sdp.primal, -sdp.dual, sdp.status)
else:
    print("Solver did not return a solution. Status:", sdp.status)

1.0000000113173735 0.99999999581288 optimal


In [11]:
Gammamon = sdp.x_mat[0]
df = pd.DataFrame(Gammamon)
print(df.to_string(float_format="{:.6f}".format))

          0         1         2         3         4         5         6         7         8         9         10        11        12
0   1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000  1.000000
1   1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000  1.000000
2  -1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000 -1.000000
3  -1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000 -1.000000
4   1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000  1.000000  1.000000
5  -1.000000 -1.000000  1.000000  1.000000 -1.000000  1.000000 -1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000 -1.000000
6   1.000000  1.000000 -1.000000 -1.000000  1.000000 -1.000000  1.000

In [12]:
Wmon = sdp.y_mat[0]
df = pd.DataFrame(Wmon)
print(df.to_string(float_format="{:.6f}".format))
np.savetxt("Wmon.csv", Wmon, delimiter=",")

          0         1         2         3         4         5         6         7         8         9         10        11        12
0   1.719284 -0.060191 -0.033566  0.843001 -0.829079 -0.107454 -0.056954 -0.042167  0.041516 -0.008910 -0.009267 -0.002571 -0.002974
1  -0.060191  1.742217 -0.013208 -0.121204 -0.051712  0.890611 -0.877065 -0.011313  0.007222 -0.006537 -0.013044 -0.000735  0.002644
2  -0.033566 -0.013208  1.728823 -0.014660  0.016481 -0.007504  0.007117  0.891826 -0.879644  0.002187  0.004436 -0.022100 -0.012898
3   0.843001 -0.121204 -0.014660  1.018432  0.024879 -0.097422  0.012698 -0.036128  0.003773 -0.068110 -0.000000  0.118780  0.000000
4  -0.829079 -0.051712  0.016481  0.024879  1.009761  0.005479  0.054059  0.001771 -0.033859  0.000000  0.051881  0.000000 -0.119950
5  -0.107454  0.890611 -0.007504 -0.097422  0.005479  1.744857  0.013840 -0.017766 -0.003701 -0.847113 -0.008473 -0.005642  0.001585
6  -0.056954 -0.877065  0.007117  0.012698  0.054059  0.013840  1.738

## Checking complementary slackness

In [13]:
Comp = Gammamon @ Wmon
df = pd.DataFrame(Comp)
print(df.to_string(float_format="{:.6f}".format))

          0         1         2         3         4         5         6         7         8         9         10        11        12
0   0.000029  0.000004  0.000014  0.000014 -0.000010  0.000009 -0.000004 -0.000001  0.000002 -0.000005  0.000000  0.000008  0.000009
1   0.000029  0.000004  0.000014  0.000014 -0.000010  0.000009 -0.000004 -0.000001  0.000002 -0.000005  0.000000  0.000008  0.000009
2  -0.000029 -0.000004 -0.000014 -0.000014  0.000010 -0.000009  0.000004  0.000001 -0.000002  0.000005 -0.000000 -0.000008 -0.000009
3  -0.000029 -0.000004 -0.000014 -0.000014  0.000010 -0.000009  0.000004  0.000001 -0.000002  0.000005 -0.000000 -0.000008 -0.000009
4   0.000029  0.000004  0.000014  0.000014 -0.000010  0.000009 -0.000004 -0.000001  0.000002 -0.000005  0.000000  0.000008  0.000009
5  -0.000029 -0.000004 -0.000014 -0.000014  0.000010 -0.000009  0.000004  0.000001 -0.000002  0.000005 -0.000000 -0.000008 -0.000009
6   0.000029  0.000004  0.000014  0.000014 -0.000010  0.000009 -0.000

Complementary slackness seems to be valid

In [ ]:
sos_mon = sdp.get_sos_decomposition()
print("SOS decomposition in the monomial basis: ", sos_mon)

**Remark:** This expression for the SOS decomposition does not seem to take into account the properties of the operators we defined. It's better to not use it!